# MongoDB

## Conexión con MongoDB y borrado de datos

In [1]:
import pymongo 
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError
import json

from pprintpp import pprint as pp

In [2]:
# connect to database
connection = MongoClient('localhost', 27017)

In [3]:
# Borramos la base de datos
connection.drop_database('black')

# Pymongo permite una sintaxis practicamente igual que la original
db = connection.black

## Lectura en Pandas

Leemos los datos desde el fichero Excel y los desnormalizamos

In [4]:
import pandas as pd
import datetime

In [5]:
df_mov = pd.read_excel("../../data/black.xlsx", sheetname= "Movimientos")
df_miembros = pd.read_excel("../../data/black.xlsx", sheetname= "Miembros")
df = pd.merge(df_mov, df_miembros, on = ['id_miembro'], how = 'inner')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7624 entries, 0 to 7623
Data columns (total 11 columns):
id_miembro            7624 non-null int64
fecha                 7624 non-null datetime64[ns]
minuto                7624 non-null int64
hora                  7624 non-null int64
importe               7624 non-null float64
comercio              6864 non-null object
actividad_completa    7622 non-null object
actividad             7622 non-null object
nombre                7624 non-null object
funcion               7624 non-null object
organizacion          6056 non-null object
dtypes: datetime64[ns](1), float64(1), int64(3), object(6)
memory usage: 714.8+ KB


## Carga en MongoDB

Gracias a Pandas convertimos el dataset en formato JSON, que es el que vamos a insertar en MongoDB prácticamente sin cambios.

In [6]:
json_string = df.to_json(orient = 'records')
json_list = json.loads(json_string)

In [7]:
pp(json_list[0])

{
    u'actividad': u'RESTAURANTE',
    u'actividad_completa': u'RESTAURANTES RESTO',
    u'comercio': u'LA VACA ARGENTINA',
    u'fecha': 1111968000000,
    u'funcion': u'concejal',
    u'hora': 16,
    u'id_miembro': 40,
    u'importe': 73.0,
    u'minuto': 11,
    u'nombre': u'José María Arteta Vico',
    u'organizacion': u'PSOE',
}


En MongoDB vamos a crear una única colección donde vamos a guardar todos los datos desnormalizados:

<img src="images/Modelo%20MongoDB.png",width=300,height=100>

<br><br> 

En la inserción de la información en mongo realizan algunas tareas de limpieza:

- Eliminar los campos nulos (no todos)
- Convertir la fecha a un formato DateTime, de forma que mongo inserte el dato correctamente
- Eliminar campos que no queremos tener en el modelo (id_miembro)

In [8]:
for movimiento_json in json_list:
    movimiento_json['fecha'] = datetime.datetime.fromtimestamp(movimiento_json['fecha'] / 1e3)
    
    del movimiento_json['id_miembro']

    if  movimiento_json['comercio'] == None:
        del movimiento_json['comercio']
    
    db.movimientos.insert_one(movimiento_json)

## Querys

Todas las querys de este caso de uso se pueden resolver directamente por la base de datos ...

### Los 10 movimientos mas caros

In [9]:
rs = db.movimientos.find({}, {
    "nombre": 1,
    "fecha": 1,
    "actividad_completa": 1,
    "importe": 1,
    "_id" : 0
}).limit(10).sort([("importe", -1)])

In [10]:
pd.DataFrame(list(rs))

,actividad_completa,fecha,importe,nombre
0,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),2006-02-14 01:00:00,11000.00,Ildefonso José Sánchez Barcoj
1,EL CORTE INGLES,2008-11-19 01:00:00,6593.20,Ildefonso José Sánchez Barcoj
2,AGENCIAS DE VIAJES,2010-11-30 01:00:00,6519.12,Matías Amat Roca
3,V.DIST.VIAJES Y TRANSPORTE DE VIAJEROS,2011-02-14 01:00:00,5500.00,María Carmen Cafranga Cavestany
4,AGENCIAS DE VIAJES,2004-01-08 01:00:00,5283.33,Ildefonso José Sánchez Barcoj
5,EL CORTE INGLES,2009-12-25 01:00:00,5000.00,Estanislao Rodríguez-Ponga Salamanca
6,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),2007-02-01 01:00:00,5000.00,Ricardo Morado Iglesias
7,AGENCIAS DE VIAJES,2010-07-07 02:00:00,4955.00,Ramón Martínez Vilches
8,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),2007-06-25 02:00:00,4500.00,Ricardo Morado Iglesias
9,EL CORTE INGLES,2010-03-25 01:00:00,4320.50,Ildefonso José Sánchez Barcoj


### Los 10 movimientos mas caros por actividad

In [11]:
rs = db.movimientos.find({
    "actividad": "HOGAR"
}, {
    "nombre": 1,
    "fecha": 1,
    "actividad_completa": 1,
    "importe": 1,
    "_id" : 0
}).limit(10).sort([("importe", -1)])

In [12]:
pd.DataFrame(list(rs))

,actividad_completa,fecha,importe,nombre
0,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",2007-10-14 02:00:00,2150.00,Miguel Ángel Araujo Serrano
1,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",2003-04-11 02:00:00,2072.00,Mariano Pérez Claver
2,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",2007-05-01 02:00:00,1510.00,Carlos María Martínez Martínez
3,"TAPICERIAS,ALFOMBRAS",2009-08-06 02:00:00,1339.94,Ricardo Romero de Tejada y Picatoste
4,MIRO ESTABLECIMIENTOS,2003-08-10 02:00:00,1198.00,Rafael Spottorno Díaz Caro
5,"ELECTRODOMESTICOS,EQUIPOS ELECTRICOS",2009-04-06 02:00:00,1000.00,Ricardo Romero de Tejada y Picatoste
6,FLORES Y PLANTAS,2008-02-14 01:00:00,1000.00,Ricardo Romero de Tejada y Picatoste
7,"ELECTRODOMESTICOS,EQUIPOS ELECTRICOS",2005-05-09 02:00:00,948.00,Alejandro Couceiro Ojeda
8,"FERRETERIA,BRICOLAJE,MENAJE DEL HOGAR",2003-01-02 01:00:00,912.00,Enrique de la Torre Martínez
9,IKEA,2011-03-27 01:00:00,835.65,Pablo Abejas Juárez


### Las 10 personas que mas han gastado

En este caso, al obtener un dato agregado, necesitamos utilizar la función **aggregate()** de mongo

In [13]:
rs = db.movimientos.aggregate([
        {"$group" : { "_id" : "$nombre", "importe" : { "$sum" : "$importe"}}},
        {"$sort" : {"importe" : -1}},
        {"$limit" : 10},
        {"$project" : {"_id" : 0, "nombre": "$_id", "importe": 1}}
    ])

In [14]:
df = pd.DataFrame(list(rs))
df

,importe,nombre
0,64590.50,Ildefonso José Sánchez Barcoj
1,47292.08,Ricardo Morado Iglesias
2,41207.26,José Antonio Moral Santín
3,39946.33,Juan Manuel Astorqui Portera
4,39820.26,Matías Amat Roca
5,39473.01,Carlos María Martínez Martínez
6,35475.15,Miguel Blesa de la Parra
7,30612.27,Ramón Ferraz Ricarte
8,26030.22,Maria Mercedes de la Merced Monge
9,25113.06,Mariano Pérez Claver


### ¿Que comercio es más popular?

Este es un caso especial de agregación, ya que cuenta los distintos. Dentro del flujo de la función se realizan varios pasos para resolver esta query:

- Formar un conjunto con los distintas personas que han comprado en cada comercio
- Desagregar el conjunto
- Agrupar los comercios, contanto los distintos elementos de cada conjunto

In [15]:
rs = db.movimientos.aggregate([
        {"$group" : { "_id" : "$comercio", "nombres" : { "$addToSet" : "$nombre"}}},
        {"$unwind" : "$nombres"},
        {"$group" : { "_id" : "$_id", "cuenta" : { "$sum" : 1}}},
        {"$sort" : {"cuenta" : -1}},
        {"$project" : {"_id" : 0, "comercio": "$_id", "cuenta": 1}},
        {"$match": {"comercio": { "$exists": True, "$ne": None }}},
        {"$limit" : 10}
    ])

In [16]:
df = pd.DataFrame(list(rs))
df

,comercio,cuenta
0,EL CORTE INGLES,52
1,IBERIA,27
2,RENFE 001,23
3,EL CORTE INGLES S.A.,19
4,VIAJES ECI,17
5,FNAC CALLAO,15
6,ACCESOS DE MADRID,13
7,PARKING EGUISA C/SEVILLA,12
8,REST EL ESPIGON,12
9,REST. MOA#A,12


Truco ...

En este tipo de querys complicadas podemos ver los resultados parciales ...

In [17]:
rs = db.movimientos.aggregate([
        {"$group" : { "_id" : "$comercio", "nombres" : { "$addToSet" : "$nombre"}}},
        {"$unwind" : "$nombres"}
    ])

pd.DataFrame(list(rs)).head()

,_id,nombres
0,MUEBLES MARIN,José Nieto Antolinos
1,SOHO,José Nieto Antolinos
2,RESTAURANTE RUGANTINO,José Luis Acero Benedicto
3,ROMANO,Enrique de la Torre Martínez
4,FOSTER S HOLLYWOOD A SORIA,José Ricardo Martínez Castro
